In [3]:
from __future__ import  print_function
import numpy as np
import pandas as pd
import sys
import re
import os

 This notebook writes hg19.cage_peak_phase1and2combined_tpm_allEntrez-sum.tsv which is a table of RLE expression values summed accross TSSs annonted with a specific entrezID

In [4]:
######################################################################################
## FUNCTION DEFINITIONS
def compareColName_with_libID(colName , libID_str):
    try:
        rval = colName.rsplit( '.' ,-2)[-2] == libID_str
    except IndexError:
        rval = False
    return rval   

def parse_FANTOMtable( fi, colIDs_list, entrezID_list ,
                              colMatchFunc =  compareColName_with_libID,
                              entrezID_column = 4,  tableSep = "\t" ,
                             checkUniqueMatch = True,
                             returnParsedHeader = True,
                             removeUnobserved_entrez = True):
    """
    Parse table of FANTOM expression values with the format:
    
    ##ColumnVariable<description_str>
    ##ColumnVariable<description_str>
    ...
    <header1>\t...<header_entrezID_col>\t... <cellType1Header>\t<cellType2Header>...
    <values1>\t ... <value2>\n ...
    ...
    Inputs:
        fi - name of the FANTOM file
        colIDs_list - list of identifers for cell type columns. the identifiers are matched with the 
                columns according by comparing element of colIDs_list with FANTOM file lines starting 
                with ##ColumnVariable. Comparison is via function colMatchFunc
        entrezID_list - only data labeled with an entrez ID from this list is added to result
        colMatchFunc - function(x,y) where x is a line of fantom file starting with ##ColumnVariable
                        and y is an entry of  colIDs_list. Returns bool
        checkUniqueMatch  - (bool) do we check that each element of colIDs_list matches only one of the
                            ##ColumnVariable... lines ?
    Returns:
        results_df - rows are entrezIDs in entrezID_list . Columns are entries of colID_list
        unobservedIDs - list of EntrezIDs that did not occur in the table
        parserdHeaderList - list of column headers from FANTOM file restricted to those columns
                            from which data was extracted. (mostly for degug purposes)
    """
    colIDs_modify = colIDs_list.copy()
    colIDs_matched = []

    ## find the column indices of the provided cell types
    colIDs_colIdx_tups = []
    f = open(fi, 'r')
    colIdx = 0
    line = f.readline()
    while line.startswith("##"):
        line = line.strip("\n")
        if not line.startswith("##ColumnVariables"):
            print("not parsing line {}".format(line) )
        else:
            matched = False
            for colID in colIDs_modify:
                if colMatchFunc(line ,   colID):
                    matched= True
                    break
            if matched:
                colIDs_colIdx_tups.append( ( colID, colIdx ) )
                colIDs_modify.remove(colID)
                colIDs_matched.append(colID)
            if checkUniqueMatch:
                if not np.all( np.logical_not([ colMatchFunc(line , x) for x in colIDs_matched ])[:-1] ):
                    print("found colIDs that matched more than once")
                    multipleMatches = [ x  for x in colIDs_matched[:-1] if colMatchFunc(line , x) ]
                    print(multipleMatches)
            colIdx+=1
        line =  f.readline()
    assert len( colIDs_modify ) == 0 , "Not all of the provided ColIDs matched"

    ## create a dataFrame to store the results
    results_df = pd.DataFrame( np.zeros(shape = (len(entrezID_list), len(colIDs_colIdx_tups)) ,dtype = float),
                                columns = [ colID for colID , _ in colIDs_colIdx_tups ] ,
                                index= entrezID_list
                             )
    ## parse the column header line to verify the mapping between column 
    ## indices and cell types
    if returnParsedHeader:
        line.strip("\n")
        colHeaders_all = line.split(tableSep)
        colHeaders = [ colHeaders_all[i] for _ ,  i in colIDs_colIdx_tups ]
    ## Parse the entries of the table
    line = f.readline()
    observedIDs = set([])
    while line:
        line = line.strip("\n")
        row = line.split(tableSep )
        entrez_entry = row[entrezID_column]
        if not entrez_entry.startswith("entrezgene:"):
            line = f.readline()
        else:
            entrezIDs = [ int(ID.strip("entrezgene:")) for ID in entrez_entry.split(",") ]
            observedIDs= observedIDs.union(set(entrezIDs))
            colValues = np.array([ float(row[i]) for _ ,  i in   colIDs_colIdx_tups ] )
            for entrezID in entrezIDs:
                try:
                    results_df.loc[entrezID, :] += colValues
                except KeyError:
                    pass
            line = f.readline()
    f.close()
    unobservedIDs = set(entrezID_list).difference(observedIDs)
    print("{} of the {} entrezIDs provided were not found in the table".format(len(unobservedIDs) , len(entrezID_list) ))
    if removeUnobserved_entrez:
        results_df.drop(list( unobservedIDs ) , axis = 0, inplace = True )

    if returnParsedHeader:
        return  results_df , unobservedIDs , colHeaders
    else:
        return results_df , unobservedIDs
    
def getEntrezIDs_FANTOMtable(fi, skipLines = 0, entrezID_column = 4,  tableSep = "\t" ,
                            debug= True , headerLine= True, returnUnique = True):

    entrezIDs_list = []
    f = open(fi, 'r')
    for i in range(skipLines):
        f.readline()
    line = f.readline()
    line = line.strip("\n")
    if debug:
        print("starting with line {}...".format(line[0:100]))
    if headerLine:
        line = f.readline() ## read past the header line
    while line:
        line =line.strip("n")
        row = line.split(tableSep )
        entrez_entry = row[entrezID_column]
        if not entrez_entry.startswith("entrezgene:"):
            line = f.readline()
        else:
            entrezIDs = [ int(ID.strip("entrezgene:")) for ID in entrez_entry.split(",") ]
            entrezIDs_list.extend(entrezIDs)
            line = f.readline()
    f.close()
    if returnUnique:
        entrezIDs_list = np.unique(entrezIDs_list)
    return entrezIDs_list
    

In [5]:
############################################################################
##  File names
fi_exprTable = "../raw/hg19.cage_peak_phase1and2combined_tpm_ann.osc.txt"
fi_cellTypes = "../raw/FANTOM5_S2Table-samples_HumanPrimaryCellAndExprAnalysis.csv"
fo = "./hg19.cage_peak_phase1and2combined_tpm_allEntrez-sum.tsv"

In [6]:
###########################################################################
## LOAD DATA
cellTypes_df = pd.read_csv(fi_cellTypes , index_col=  0, header= 0)

In [8]:
## get an array of all the unique entrezIDs in the FANTOM table
entrezID_unique = getEntrezIDs_FANTOMtable(fi = fi_exprTable, skipLines =1837, 
                                           entrezID_column = 4,  tableSep = "\t" ,
                                            debug= True , headerLine= True, 
                                           returnUnique = True)

starting with line 00Annotation	short_description	description	association_with_transcript	entrezgene_id	hgnc_id	uniprot...


In [10]:
##########################################################################################
### parse the FANTOM table to a (num_entrezID , num_cellTypes) dataframe 
#### where each entry is the summed tpm value of all CAGE peaks associated with
##### given entrezID
results_df , unobservedIDs , colHeaders = parse_FANTOMtable( fi = fi_exprTable, 
                                                colIDs_list =list(cellTypes_df.index.values), 
                                                entrezID_list = list(entrezID_unique ), 
                                                colMatchFunc = compareColName_with_libID,
                                                entrezID_column = 4, 
                                                tableSep = "\t" ,
                                                checkUniqueMatch = True,
                                                returnParsedHeader = True,
                                                removeUnobserved_entrez = True)

not parsing line ##ParemeterValue[genome_assembly]=hg19
0 of the 19088 entrezIDs provided were not found in the table


In [11]:
## label the index as entrezID and modify column names to include cell type info
results_df.index.name ="entrezID"
results_df.rename(mapper = lambda x: cellTypes_df.loc[x, "description"] + "-{}".format(x)  , 
                        axis = 'columns' , inplace = True)

## check the the columns of FANTOM table we extracted data from match the columns of results_df
assert len(colHeaders) ==len(results_df.columns ) 
assert(np.all( [compareColName_with_libID(x, y.rsplit("-",-1)[-1]) \
                for x , y in zip( colHeaders ,  results_df.columns  ) ] ) )

In [12]:
results_df.to_csv(fo + ".gz", sep = "\t" ,  compression= "gzip")

In [13]:
results_df.head(n=3)

,"Adipocyte - breast, donor1-CNhs11051","Adipocyte - breast, donor2-CNhs11969","Adipocyte - omental, donor1-CNhs11054","Adipocyte - omental, donor2-CNhs12067","Adipocyte - omental, donor3-CNhs12068","Adipocyte - perirenal, donor1-CNhs12069","Adipocyte - subcutaneous, donor1-CNhs12494","Adipocyte - subcutaneous, donor2-CNhs11371","Adipocyte - subcutaneous, donor3-CNhs12017","Alveolar Epithelial Cells, donor1-CNhs11325",...,"migratory langerhans cells, donor2-CNhs13536","migratory langerhans cells, donor3-CNhs13547","nasal epithelial cells, donor1-CNhs12589","nasal epithelial cells, donor2-CNhs12574","salivary acinar cells, donor1-CNhs12810","salivary acinar cells, donor2-CNhs12811","salivary acinar cells, donor3-CNhs12812","tenocyte, donor1-CNhs12639","tenocyte, donor2-CNhs12640","tenocyte, donor3-CNhs12641"
entrezID,,,,,,,,,,,,,,,,,,,,,
1,50.143233,23.551058,26.499428,41.030052,37.991693,49.723296,48.126977,38.534912,36.565988,2.703073,...,22.719888,33.040210,7.553493,13.063622,14.919564,3.926525,5.530077,23.807681,7.856986,24.659714
2,124.449690,56.094337,235.957672,84.061571,44.249148,50.595634,83.588960,99.811740,6.790826,0.000000,...,147.479973,90.615228,0.000000,0.622077,0.621648,4.267962,2.765039,3.347955,12.016566,0.432627
9,9.265597,4.067910,10.289671,8.005864,5.810494,9.595724,5.065998,5.053759,1.828299,7.372018,...,57.663341,51.359534,2.098193,0.622077,2.486594,1.877903,1.382519,3.719950,3.697405,6.056772
